In [1]:
import spacy
from spacy.scorer import Scorer
from spacy.training import Example
from spacy.tokens import DocBin
import pandas as pd

In [ ]:
spacy.require_gpu()
vac_nlp = spacy.load(r"output_v4\model-best")

c:\Users\wpoto\.conda\envs\use_spacy\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.8.7 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\wpoto\.conda\envs\use_spacy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\wpoto\.conda\envs\use_spacy\Lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle d

In [3]:
def vac_identification(text, vac_nlp):
    vac_doc = vac_nlp(text)
    
    token_list = []
    label_list = []

    data = []
    data_list = []

    for token in vac_doc:
        if token.text != "\n":
            token_list.append(token.text)
            label_list.append(token.ent_type_)
        if token.ent_type_:
            data.append({
                "text": token.text,
                "ent_type": token.ent_type_
            })
            data_list.append((token.text, token.ent_type_))

    df = pd.DataFrame({"Token": token_list, "VAC_label": label_list})
    return df, data_list

サンプルテキスト

In [ ]:
text = """
The new employee seemed nervous during his first presentation.
He talked about his previous experience and described what he wanted to achieve.
The manager found his ideas interesting and encouraged him to develop them further.
"""

df, data_list = vac_identification(text, vac_nlp)

display(df)

data_list

,Token,VAC_label
0,The,
1,new,
2,employee,
3,seemed,V_ADJ
4,nervous,
5,during,
6,his,
7,first,
8,presentation,
9,.,


[('seemed', 'V_ADJ'),
 ('talked', 'V_ABOUT_N'),
 ('described', 'V_WH'),
 ('wanted', 'V_TO_INF'),
 ('achieve', 'V_N_OBJ'),
 ('found', 'V_N_ADJ'),
 ('encouraged', 'V_N_TO_INF'),
 ('develop', 'V_N_OBJ')]

表層が同じでパターンが異なる文（前置詞）：

In [ ]:
text = """
He has been playing the guitar for 10 years.
Don’t take me for a fool.
"""

df, data_list = vac_identification(text, vac_nlp)
display(df)

data_list

,Token,VAC_label
0,He,
1,has,
2,been,
3,playing,V_N_OBJ
4,the,
5,guitar,
6,for,
7,10,
8,years,
9,.,


[('playing', 'V_N_OBJ'), ('take', 'V_N_FOR_N')]

表層が同じでパターンが異なる（名詞修飾）：

In [ ]:
text = """
I saw the man crossing the street.
She chose a dress matching your tie.
"""

df, data_list = vac_identification(text, vac_nlp)
display(df)

data_list

,Token,VAC_label
0,I,
1,saw,V_N_ING
2,the,
3,man,
4,crossing,V_N_OBJ
5,the,
6,street,
7,.,
8,She,
9,chose,V_N_OBJ


[('saw', 'V_N_ING'),
 ('crossing', 'V_N_OBJ'),
 ('chose', 'V_N_OBJ'),
 ('matching', 'V_N_OBJ')]

表層が同じでパターンが異なる（不定詞）

In [7]:
text = """
I want Saki to buy me a new car.
I studied English to earn money.
"""

df, data_list = vac_identification(text, vac_nlp)
display(df)
data_list

,Token,VAC_label
0,I,
1,want,V_N_TO_INF
2,Saki,
3,to,
4,buy,V_N_N_OBJ
5,me,
6,a,
7,new,
8,car,
9,.,


[('want', 'V_N_TO_INF'),
 ('buy', 'V_N_N_OBJ'),
 ('studied', 'V_N_OBJ'),
 ('earn', 'V_N_OBJ')]

誤りを含む文

In [8]:
text = """
I enjoyed to eat the food.
I enjoyed eating the food.
"""

df, data_list = vac_identification(text, vac_nlp)
display(df)
data_list

,Token,VAC_label
0,I,
1,enjoyed,V_TO_INF
2,to,
3,eat,V_N_OBJ
4,the,
5,food,
6,.,
7,I,
8,enjoyed,V_ING
9,eating,V_N_OBJ


[('enjoyed', 'V_TO_INF'),
 ('eat', 'V_N_OBJ'),
 ('enjoyed', 'V_ING'),
 ('eating', 'V_N_OBJ')]